# The Helmotz decomposition for a function in $[( \mathbb{R}) ]^3$
The problem now becomes 2 separate minimization problem:
$$
\begin{cases}
\min_{\psi\in H(curl)} \| F - curl(\psi)\|^2_{L_2} \\

div\psi = 0\\
\end{cases}
$$
and 
$$
\begin{cases}
\min_{\phi\in H(div)} \| F - \nabla \phi \|^2_{L_2} \\
\int\phi = 0\\
\end{cases}
$$

The above problem can now be minimized using the variational formulation of the problem (... we create the lagrangian functional and we set the first variation to zero ...).


The first problem becomes:

Find $(\psi , p) \in H(\text{curl})\times H(\text{div})$ s.t. 
$$
\begin{cases}
(\text{curl }\psi ,\text{curl }\hat\psi) + (\text{div } \psi, p) = (\text{curl }\hat\psi , F) \\
(\text{div }\hat\psi, \hat{p} ) = 0 \\
\end{cases}
$$
For all $(\hat\psi , \hat{p}) \in H(\text{curl})\times H(\text{div})$ 

The second problem becomes:

Find $(\phi , u) \in H^1\times \mathbb{R}$ s.t. 
$$
\begin{cases}
(\nabla\phi ,\nabla\hat\phi) + (\nabla \phi, u) = (\nabla\hat\phi , F) \\
(\nabla\hat\phi, \hat{u} ) = 0 \\
\end{cases}
$$
For all $(\hat\phi , \hat{u}) \in  H^1\times \mathbb{R}$




In [1]:
from ngsolve import * # needed for all the tools
from ngsolve.krylovspace import CGSolver # when we have to use a conjugate gradient solver
from ngsolve.webgui import Draw # needed jupiter visualization

#from ngsolve.meshes import MakeStructured3DMesh
def L2Norm(u, mesh):
    return sqrt(Integrate(InnerProduct(u,u), mesh))

def Curl(u):
    # calculate the curl of u with u = gf_u or u = cf_u
    if u.dim == 3:
        return CF( (u[2].Diff(y)- u[1].Diff(z), u[0].Diff(z)- u[2].Diff(x), u[1].Diff(x)- u[0].Diff(y)) )
    if u.dim == 9:
        return CF( (Curl(u[0,:]),Curl(u[1,:]),Curl(u[2,:])),dims=(3,3) )   

def Div(u):
    # calculate the divergence of u with u = gf_u or u = cf_u
    return CF( (u[0].Diff(x) + u[1].Diff(y) + u[2].Diff(z)) )
    


def HelmholtzDecomposition(F, mesh, order=3, grad_phi=None, curl_psi=None, draw=False , moreinfo = False):
    # q : is the function to decompose, can be passed as CoefficientFunction or as gridfunction.
    # mesh : is the mesh on which the decomposition is done.
    # order = 3 : is the minimum order of the finite element space we involve in the decomposition
    # grad_phi = None : is the gradient parti of the function to decompose. if it's known, it can be passed as CoefficientFunction or as gridfunction.
    # curl_psi = None : is the curl part of the function to decompose. if it's known, it can be passed as CoefficientFunction or as gridfunction.
    # draw = False : is a boolean that decides if to plot the results.
    # moreinfo = False : is a boolean that decides if to print more information, i.e. the norms of the decomposition.
    # returns : the decomposition of the function q in the following gridfunctions.
    #           gf_


    # control parameters
    if order < 1:
        raise Exception("Wrong order!")

    ##########################
    #   determine curl part  #
    ##########################
    
    # define the FES :
    #
    # Note: we use different order for the shape functions since 
    #       in the mass matrix "a" have Grad(q)*u and they need 
    #       to be of the same order.
    fes = HCurl(mesh, order=order, dirichlet=".*")*H1(mesh,order=order+1, dirichlet=".*")
    (u,p), (v,q) = fes.TnT()

    # Linear and Bilinear form 
    a = BilinearForm(fes, symmetric=True, symmetric_storage=True, condense=False)
    a += (curl(u)*curl(v) + Grad(p)*v + Grad(q)*u)*dx
        
    f = LinearForm(fes)
    f += F*curl(v)*dx

    a.Assemble()
    f.Assemble()

    inv = a.mat.Inverse(fes.FreeDofs(a.condense), inverse="pardiso")

    # grid function to store the solution
    #
    # note : the gridfunction is has 2 components,
    #        one for the curl and one for the divergence. 
    #        we need to split it
    gf_sol = GridFunction(fes)
    gf_psi, gf_p = gf_sol.components

    # Solution of the system
    #
    # note 1: we use a possibility to condense the matrix and to use a harmonic extension
    #        to get a better convergence speed ... more on this topic in the tutorial  
    #        https://docu.ngsolve.org/v6.2.1810/i-tutorials/unit-1.4-staticcond/staticcond.html?highlight=condense
    #
    # note 2: the flag for condense is set to False by default. 
    r = f.vec.CreateVector()
    w = f.vec.CreateVector()       
    r.data = f.vec
    if a.condense:
        r.data += a.harmonic_extension_trans * r
    w.data = inv * r
    if a.condense:
        w.data += a.harmonic_extension * w
        w.data += a.inner_solve * r
    gf_sol.vec.data = w

    # the curl we find is defined as the curl of the gridfunction gf_psi
    gf_curl_psi = CF( (Grad(gf_psi)[2,1]-Grad(gf_psi)[1,2],Grad(gf_psi)[0,2]-Grad(gf_psi)[2,0],Grad(gf_psi)[1,0]-Grad(gf_psi)[0,1]) )


    # draw flag allows to draw the results
    #
    # note : the flag is set to False by default.
    if draw:
        print("gf_psi")
        Draw(gf_psi, mesh, "gf_psi", clipping=(1,0,0,0))

        print("gf_curl_psi")
        Draw(BoundaryFromVolumeCF(gf_curl_psi), mesh, "gf_curl_psi",clipping=(1,0,0,0))

    ##########################
    #   determine grad part  #
    ##########################

    # define the FES :
    #
    # Note: we don't need an order specified for the NumberSpace.
    fes2 = H1(mesh,order=order)*NumberSpace(mesh)
    (u,p),(v,q) = fes2.TnT()
    a2 = BilinearForm(fes2, symmetric=True, symmetric_storage=True)
    a2 += (Grad(u)*Grad(v) + p*v + q*u)*dx

    f2 = LinearForm(fes2)
    f2 += F*Grad(v)*dx

    a2.Assemble()
    f2.Assemble()

    # grid function to store the solution
    gf_sol2 = GridFunction(fes2)
    gf_phi, gf_N = gf_sol2.components

    # Solution of the system
    gf_sol2.vec.data = a2.mat.Inverse(fes2.FreeDofs(), inverse="pardiso")*f2.vec

    # the gradient we find is defined as the gradient of the gridfunction gf_phi
    gf_grad_phi= Grad(gf_phi)

    # Drawings and informations:

    #for the sclar potential we have
    if draw:
        print("the scalar potential gf_phi")
        Draw(gf_phi, mesh, "gf_phi",clipping=(1,0,0,0))
    if moreinfo and (grad_phi is not None):
        if draw :
            print("the gradient of the scalar potential gf_phi : ")
            Draw(BoundaryFromVolumeCF(gf_grad_phi), mesh, "gf_grad_phi",clipping=(1,0,0,0))
        print("the gradient of the scalar potential gf_phi - the known gradient : ")
        print("the L2-error of gf_grad_phi-grad_phi : ", L2Norm(gf_grad_phi-grad_phi, mesh))
        if draw :
            Draw(BoundaryFromVolumeCF(gf_grad_phi-grad_phi), mesh, "gf_grad_phi - grad_phi",clipping=(1,0,0,0))
    
    #for the vector potential we have
    if draw:
        print("the vector potential gf_psi")
        Draw(gf_psi, mesh, "gf_psi",clipping=(1,0,0,0))
    if moreinfo and (curl_psi is not None):
            if draw :
                print("the curl of the vector potential gf_psi : ")
                Draw(BoundaryFromVolumeCF(gf_curl_psi), mesh, "gf_curl_psi",clipping=(1,0,0,0))
            print("the curl of the vector potential gf_psi - the known curl : ")
            print("the L2-error is : ", L2Norm(gf_curl_psi-curl_psi, mesh))
            if draw:
                Draw(BoundaryFromVolumeCF(gf_curl_psi-curl_psi), mesh, "gf_curl_psi - curl_psi",clipping=(1,0,0,0))

    print("the solution gf_q_h = gf_grad_phi + ") 

    if draw:
        #print(" err q: ", sqrt(Integrate((Grad(gf_phi)+gf_curl_psi-F)**2,mesh)))
        #Draw(Grad(gf_phi)+gf_curl_psi-F, mesh)
        print(" err q: ", sqrt(Integrate((Grad(gf_phi)+gf_curl_psi-F)**2,mesh)))
        Draw(gf_grad_phi+gf_curl_psi-F, mesh,clipping=(1,0,0,0))

        if grad_phi and moreinfo :
            print("err grad_phi: ", sqrt(Integrate((Grad(gf_phi)-grad_phi)**2,mesh)))
            Draw(Grad(gf_phi)-grad_phi, mesh,clipping=(1,0,0,0))
        if curl_psi  and moreinfo :
            print("err curl_psi: ", sqrt(Integrate((gf_curl_psi-curl_psi)**2,mesh)))
            Draw(gf_curl_psi-curl_psi, mesh,clipping=(1,0,0,0))

        
    return gf_psi, gf_phi, gf_curl_psi , gf_grad_phi



# One Simple Example
One notice that the decomposition is not perfect, since when someone defines a function $F = \nabla \phi  + \nabla \times \psi$ has to keep in mind how the process of decomposition for $F \in H^1$ takes place:

the first step is to solve the neumann problem $\Delta \phi = \nabla \cdot F$ with $\phi \in H(div)$ therefore if $\nabla \cdot F$ has also a tangential component on the boundary in $H^1$ then that component is not approximated.

The second step works in a similar way but for the normal component of $\nabla \times F$

For more info look at the thesis of J. Schoeberl's notes on Maxwell's equations

Time of execution 20 seconds

In [4]:
from netgen.csg import unit_cube # allow us to create a unit cube in one line of code


# define a mesh
moreinfo = True
draw1 = True   # draw flag defined in the HelmholtzDecomposition(... ,draw = draw1 ,....)
draw2 = True # used to visualize the definition of the function F
MaxH = 0.15 # maximum mesh size
mesh = Mesh(unit_cube.GenerateMesh(maxh=MaxH)) 
order = 4 

    ##############################################################
    #  Create a function F = grad(phi) + curl(psi) from scratch  #
    ############################################################## 

# Scalar Potential phi
cf_phi = (1-x**2)**2*(1-y**2)**2*(1-z**2)**2 
if draw2: print("cf_phi") , Draw(cf_phi, mesh) 
    

# define the irrotational Part of function
cf_grad_phi = CF( (cf_phi.Diff(x),cf_phi.Diff(y),cf_phi.Diff(z)) ) 
if draw2: print("cf_grad_phi") ,  Draw(cf_grad_phi, mesh)

# Vector potential A, 
cf_psi = (1-x**2)*(1-y**2)*CF( (0,0,1) )
if draw2 : print("cf_psi"),Draw(cf_psi, mesh)


# divergence of A .. has to be zero!!!
# if you want to check this, uncomment the following line
#
#cf_div_psi = Div(cf_psi)
#if draw2: print("cf_div_psi") ,  Draw(cf_div_psi, mesh)

# Rotatioal part of the function
cf_curl_psi = Curl(cf_psi)
if draw2: print("cf_curl_psi"), Draw(cf_curl_psi, mesh)

# Function We are trying to decompose
F = cf_grad_phi+cf_curl_psi
if draw2: print("F"),  Draw(F, mesh)

cf_phi


WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, 'draw_vol': True…

cf_grad_phi


WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, 'draw_vol': True…

cf_psi


WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, 'draw_vol': True…

cf_curl_psi


WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, 'draw_vol': True…

F


WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, 'draw_vol': True…

In [3]:
# It is better to define the taskmanager outside the function, otherwise it will be created every time we call the function,
# also one could be interested in the performance of the function.

with TaskManager():
    gf_psi, gf_phi, gf_curl_psi , gf_grad_phi = HelmholtzDecomposition(F, mesh, order=order, grad_phi=cf_grad_phi, curl_psi=cf_curl_psi, draw=draw1, moreinfo=moreinfo)

gf_psi


WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, 'draw_vol': True…

gf_curl_psi


WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, 'draw_vol': True…

the scalar potential gf_phi


...

WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, 'draw_vol': True…

err grad_phi:  1.0679777198026765


WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, 'draw_vol': True…

err curl_psi:  1.0679779872566393


WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, 'draw_vol': True…

Some comments on the previous function: As we see setting the flag on **moreinfo** and **draw** to True it shows that there is a problem with the $L_2$-norm on the boundary.

This is caused by the fact that the boundary is homogeneous Dirichlet only on the vectorial Potential and Neumann on the scalar Potential. Therefore the discrepancy between the two norms of the 2 components of $F$.


In [42]:
### this part to revisit ###


# define a function that checks the norm of the function cf_grad_phi in the normal direction
def CheckNorm_normal(cf_u, mesh):
    # define the normal direction
    n = specialcf.normal(mesh.dim)
    # define the normal component of the gradient of the scalar potential
    cf_u_n = InnerProduct(cf_u, n)
    # define the L2-norm of the normal component of the gradient of the scalar potential
    return L2Norm(cf_u_n, mesh)

# define a function that checks the norm of the function cf_grad_phi in the normal direction
def CheckNorm_tangential(cf_u, mesh):
    # define the normal direction
    n = specialcf.normal(mesh.dim)
    # define the normal component of the gradient of the scalar potential
    cf_u_t = cf_u - InnerProduct(cf_u, n)*n
    # define the L2-norm of the normal component of the gradient of the scalar potential
    return L2Norm(cf_u_t, mesh)

# the error in the normal and tangential direction, we expect to see some difference in the tangential direction
print(CheckNorm_tangential(gf_grad_phi, mesh))
print(CheckNorm_normal(gf_grad_phi, mesh))
print(L2Norm(gf_grad_phi-cf_grad_phi, mesh))

#print(L2Norm(gf_grad_phi-cf_grad_phi, mesh))

print(CheckNorm_tangential(gf_curl_psi, mesh))
print(CheckNorm_normal(gf_curl_psi, mesh))
print(L2Norm(gf_curl_psi-cf_curl_psi, mesh))


print(CheckNorm_tangential(F, mesh))
print(CheckNorm_normal(F, mesh))
print(L2Norm(gf_grad_phi+gf_curl_psi-F, mesh))

1.3207734011657788
0.0
1.067977719802677
0.5307021467034689
0.0
1.0679779872566408
1.4234073181222027
0.0
0.0005259161442585453
